In [9]:
# 包导入
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import random

In [16]:
# 数据生成的参数设置
# #
hour_rate = [0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 
             0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1,0.2, 0.3]
people = 1000
down_flow = 5000.0
up_flow = 100.0
file_path = "data.tfrecord"


# 写数据到TFrecord文件
# #
# 生成首个数据，接下来的数据在前23小时上加一小时
first_day = np.zeros(120)
for i in range(0, 24):
    avg_people = hour_rate[i%24] * random.randint(30,80) * people / 10 
    max_people = avg_people * random.randint(20, 40) / 10
    down_f = hour_rate[i%24] * random.randint(30,80) * down_flow / 10
    up_f = hour_rate[i%24] * random.randint(30, 80) *  up_flow / 10
    first_day[i*5] = avg_people
    first_day[i*5+1] = max_people
    first_day[i*5+2] = up_f
    first_day[i*5+3] = down_f
    first_day[i*5+4] = (up_f + down_f)/2


# 写入数据
with tf.io.TFRecordWriter(file_path) as files:
    # 第一个数据的写入
    example_first = tf.train.Example(
        features=tf.train.Features(
            feature={
                "lable":tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[b"avg,max,up,down,all"])),
                "data": tf.train.Feature(
                    float_list=tf.train.FloatList(value=first_day))
                }))
    for i in range(2):
        files.write(example_first.SerializePartialToString())



In [17]:
# 读数据函数,返回解析后的结果
def read_datas(filepath):
    feature_description = {
        "lable":tf.io.VarLenFeature(dtype=tf.string),
        "data": tf.io.VarLenFeature(dtype=float)
    }

    file_dataset = tf.data.TFRecordDataset(filepath).batch(2)

    for serialized_example in file_dataset:
        parsed_example = tf.io.parse_example(serialized_example,                                                                                                       feature_description)
    
    return parsed_example

SyntaxError: invalid syntax (<ipython-input-17-d1b49743a496>, line 5)

In [21]:
datas = read_datas(file_path)["data"].values
for i in datas:
    print(i)

tf.Tensor(480.0, shape=(), dtype=float32)
tf.Tensor(1728.0, shape=(), dtype=float32)
tf.Tensor(33.0, shape=(), dtype=float32)
tf.Tensor(1550.0, shape=(), dtype=float32)
tf.Tensor(791.5, shape=(), dtype=float32)
tf.Tensor(1280.0, shape=(), dtype=float32)
tf.Tensor(2816.0, shape=(), dtype=float32)
tf.Tensor(80.0, shape=(), dtype=float32)
tf.Tensor(4000.0, shape=(), dtype=float32)
tf.Tensor(2040.0, shape=(), dtype=float32)
tf.Tensor(1620.0, shape=(), dtype=float32)
tf.Tensor(5832.0, shape=(), dtype=float32)
tf.Tensor(195.0, shape=(), dtype=float32)
tf.Tensor(10500.0, shape=(), dtype=float32)
tf.Tensor(5347.5, shape=(), dtype=float32)
tf.Tensor(310.0, shape=(), dtype=float32)
tf.Tensor(1178.0, shape=(), dtype=float32)
tf.Tensor(70.0, shape=(), dtype=float32)
tf.Tensor(3300.0, shape=(), dtype=float32)
tf.Tensor(1685.0, shape=(), dtype=float32)
tf.Tensor(1140.0, shape=(), dtype=float32)
tf.Tensor(2964.0, shape=(), dtype=float32)
tf.Tensor(80.0, shape=(), dtype=float32)
tf.Tensor(4400.0, shap

In [22]:
a = np.array([1, 2, 3])
b = np.array(10)
b[1:3] = a
print(b)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [ ]:
# CNN模块创建
# 步频为1，输入输出的空间维度不变[24,4]
udn_model = keras.models.Sequential([
    keras.layers.Conv2D(128, 2, activation="relu", padding="same",
                         input_shape=[24, 4, 1]),
    keras.layers.Conv2D(64, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(16, 2, activation="relu", padding="same"),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="relu")
])

udn_model.summary()

In [ ]:
# 数据处理
# 验证小区上的流量，标签为小区下一个小时流量
# y_true为小区下个小时的流量
# 数据集为[24,4]的张量，将([i,3]+[i,4])=y_train[]
# X_train[i] --> y_train[i+1]
# 由于tf的reshape只接受32的大小
# 所以用numpy进行数据的构造，并用np.reshap()进行构造
# # 
    # 接下来的数据生成并输入
    pre_day = first_day
    for i in range (0, 5):
        next_day = np.zeros(120)
        next_day[0:114] = pre_day[5:119]

        avg_people = hour_rate[i] * random.randint(30,80) * people / 10 
        max_people = avg_people * random.randint(20, 40) / 10
        down_f = hour_rate[i] * random.randint(30,80) * down_flow / 10
        up_f = hour_rate[i] * random.randint(30, 80) *  up_flow / 10
        next_day[115] = avg_people
        next_day[116] = max_people
        next_day[117] = up_f
        next_day[118] = down_f
        next_day[119] = (up_f + down_f)/2

        # 写入数据
        example_next = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "data": tf.train.Feature(
                        float_list=tf.train.FloatList(value=next_day))
                    }))

        files.write(example_next.SerializePartialToString())
        # 更新数据
        pre_day = next_day

In [ ]:
# 首个数据写入
# TFrecord二次读写会覆盖
hour_rate = [0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2,              0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1,               0.2, 0.3,]
people = 1000
down_flow = 5000.0
up_flow = 100.0
file_path = "datas.tfrecord"

one_day = np.zeros(96)
for i in range(0, 24):
    avg_people = hour_rate[i] * random.randint(30,80) * people / 10 
    max_people = avg_people * random.randint(20, 40) / 10
    down_f = hour_rate[i] * random.randint(30,80) * down_flow / 10
    up_f = hour_rate[i] * random.randint(30, 80) *  up_flow / 10
    one_day[i*4] = avg_people
    one_day[i*4+1] = max_people
    one_day[i*4+2] = up_f
    one_day[i*4+3] = down_f

with tf.io.TFRecordWriter(file_path) as files:
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "data": tf.train.Feature(
                    float_list=tf.train.FloatList(value=one_day))
                }))
    files.write(example.SerializePartialToString())

In [ ]:
# 读数据函数,返回解析后的结果
def read_datas(filepath):
    feature_description = {
        "data": tf.io.VarLenFeature(dtype=float)
    }

    file_dataset = tf.data.TFRecordDataset(filepath).batch(2)

    for serialized_example in file_dataset:
        parsed_example = tf.io.parse_example(serialized_example,                                                  feature_description)
    
    return parsed_example

In [ ]:
# 读取数据并输出
datas = read_datas(file_path)
print(datas["data"].values)

In [ ]:
# 损失函数为负对数似然（negative log likelihood）
def negative_log_likelihood(y_true, y_pred, eps=1e-15):
    p = keras.backend.clip(y_pred, eps, 1-eps)
    loss = keras.backend.sum(-y_true * keras.backend.log(p) -                                    (1-y_true) * keras.backend.log(1-p))
    
    return keras.backend.mean(loss)

In [ ]:
# 编译模块
SGD = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
udn_model.compile(
    loss=negative_log_likelihood,
    optimizer=SGD
)

In [ ]:
# 训练模块
# 在训练时运用mini-batch方法进行小批量梯度、
# 对每个小时进行预测
udn_model.fit(
    x=X_train
    y=y_true
    batch_size=5
    epochs=1
    callbacks="EarlyStopping"
)

In [ ]:
# 验证模块
udn_model.evaluate(

)

In [ ]:
# 进行预测
udn_model.predict(
    
)